In [28]:
import random
import string
import os

In [29]:
def generate_random_files(byte_size: int):
    random_text: str = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + ' ', k = byte_size))
    return random_text

def write_to_file(directory: str, filename: str, byte_size: int):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory,filename)
    random_text = generate_random_files(byte_size)

    with open(file_path,'w') as file:
        file.write(random_text)

In [30]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from binascii import hexlify
from numpy import array, mean, var, std, sqrt

import timeit, secrets

In [31]:
def aes_encrypt_file(path: str, key: int, new_path: str = None) -> tuple[int, int]:
    # Generate a random IV (16 bytes for AES)
    iv = secrets.token_bytes(16)

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    # Read the plaintext file in binary mode
    with open(path, "rb") as plaintext_file:
        plaintext = plaintext_file.read()

    # Make sure the plaintext length is a multiple of block size (AES block size is 16 bytes)
    padding_length = 16 - len(plaintext) % 16
    padded_plaintext = plaintext + bytes([padding_length]) * padding_length

    # Encrypt the padded plaintext
    ct = encryptor.update(padded_plaintext) + encryptor.finalize()

    # Write the ciphertext to a new file
    with open(path if new_path == None else new_path + ".bin", "wb") as cphFile:
        cphFile.write(iv)  # Prepend IV to the ciphertext for decryption later
        cphFile.write(ct)

    # print(f"Encryption successful! File saved as: {new_path or path}")
    return hexlify(key), iv

def aes_decrypt_file(path: str, key: int, new_path: str = None) -> None:
    # Read the ciphertext file
    with open(path, "rb") as cphFile:
        iv = cphFile.read(16)  # Extract the IV (first 16 bytes)
        ct = cphFile.read()    # Read the rest as ciphertext

    # Create the cipher
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    padded_plaintext = decryptor.update(ct) + decryptor.finalize()

    # Remove padding (last byte value determines padding length)
    padding_length = padded_plaintext[-1]
    plaintext = padded_plaintext[:-padding_length]

    # Write the decrypted plaintext to a new file
    with open(path[:-4] if new_path is None else new_path, "wb") as plaintext_file:
        plaintext_file.write(plaintext)

    # print(f"Decryption successful! File saved as: {new_path or path}")

In [32]:
def test_aes(files: str, times: int, repeat: int) -> tuple[tuple[float, float, float], tuple[float, float, float]]:
    results: list[list[float]] = [[], []]
    for file in files:
        key: int = os.urandom(32)
        encrypt_trials = array(timeit.repeat(
                lambda: aes_encrypt_file(
                    file,
                    key,
                    new_path = os.path.join(os.path.dirname(file), "encrypted_" + os.path.basename(file))
                ),
                number = times,
                repeat = repeat
            ))
        decrypt_trials = array(timeit.repeat(
                lambda: aes_decrypt_file(
                    os.path.join(os.path.dirname(file), "encrypted_" + os.path.basename(file) + ".bin"),
                    key,
                    new_path = (os.path.join(os.path.dirname(file), "decrypted_" + os.path.basename(file)))
                ), # because the encryption function turns the key into binary, unhexlify(key) would be called here to undo it
                number = times,
                repeat = repeat
            ))
        results[0].extend(encrypt_trials)
        results[1].extend(decrypt_trials)

    # Compute mean of means
    mean_encrypt = mean(results[0])
    mean_decrypt = mean(results[1])

    # Compute pooled variance
    var_encrypt = var(results[0], ddof = 1)
    var_decrypt = var(results[1], ddof = 1)

    # Compute pooled standard error
    stderr_encrypt = std(results[0], ddof = 1) / sqrt(len(results[0]))
    stderr_decrypt = std(results[1], ddof = 1) / sqrt(len(results[1]))

    return (
        (mean_encrypt, var_encrypt, stderr_encrypt),
        (mean_decrypt, var_decrypt, stderr_decrypt)
    )

In [33]:
def print_results(test: str, result: tuple[tuple[float, float, float], tuple[float, float, float]]) -> None:
    print(f"Test {test}:\n\t")
    print(f"encryption:\n\t\tmean: {result[0][0]:.5f}\n\t\tstd error: {result[0][1]:.5e}\n\t\tvariance: {result[0][2]:.5e}")
    print(f"decryption:\n\t\tmean: {result[1][0]:.5f}\n\t\tstd error: {result[1][1]:.5e}\n\t\tvariance: {result[1][2]:.5e}")

In [35]:
sizes: int = [8, 64, 512, 4096, 32768, 262144, 2097152]
directory: str = 'random_files'

amount: int = 10
for size in sizes:
    for i in range(amount):
        filename: str = f'random_text_{size}_{i}.txt'
        write_to_file(directory, filename, size)

files: list[str] = [[directory + f"/random_text_{size}_{i}.txt" for i in range(amount)] for size in sizes]
for i in range(len(sizes)):
    print_results(sizes[i], test_aes(files[i], 100, 1))

for i in range(len(sizes)):
    for file in files[i]:
        if os.path.isfile(file):
            os.remove(file)
            os.remove(os.path.join(os.path.dirname(file), "encrypted_" + os.path.basename(file) + ".bin"))
            os.remove(os.path.join(os.path.dirname(file), "decrypted_" + os.path.basename(file)))

Test 8:
	
encryption:
		mean: 0.01019
		std error: 5.70385e-06
		variance: 7.55238e-04
decryption:
		mean: 0.00911
		std error: 2.60658e-07
		variance: 1.61449e-04
Test 64:
	
encryption:
		mean: 0.00832
		std error: 1.48251e-07
		variance: 1.21758e-04
decryption:
		mean: 0.00834
		std error: 2.43385e-07
		variance: 1.56008e-04
Test 512:
	
encryption:
		mean: 0.00838
		std error: 8.91280e-08
		variance: 9.44076e-05
decryption:
		mean: 0.00825
		std error: 1.02830e-07
		variance: 1.01405e-04
Test 4096:
	
encryption:
		mean: 0.00978
		std error: 4.45148e-08
		variance: 6.67194e-05
decryption:
		mean: 0.00842
		std error: 3.83441e-08
		variance: 6.19226e-05
Test 32768:
	
encryption:
		mean: 0.01590
		std error: 5.91990e-07
		variance: 2.43308e-04
decryption:
		mean: 0.01303
		std error: 5.95026e-07
		variance: 2.43932e-04
Test 262144:
	
encryption:
		mean: 0.05852
		std error: 1.19022e-05
		variance: 1.09097e-03
decryption:
		mean: 0.03604
		std error: 6.33017e-06
		variance: 7.95624e-04
T